# Importing Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim  # optimizer used to update the weights

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)

# Checking the GPU availability and properties

In [2]:
# Check for detecting if GPU is available
print(torch.__version__)
print(torchvision.__version__)
print(torch.cuda.is_available())

# GPU device details
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

1.7.0+cu101
0.8.1+cu101
True
0
1
Tesla T4


# E-T-L Step

Extract ==> Transform ==> Load

In [3]:

# Extracting/Downloading and transforming the images to tensor
train_set = torchvision.datasets.EMNIST(
    root='./data/EMNIST'
    ,split='byclass'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

# Loading the training images in the form of batches.
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

Extracting ./data/EMNIST/EMNIST/raw/emnist.zip to ./data/EMNIST/EMNIST/raw
Processing byclass


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Processing bymerge
Processing balanced
Processing letters
Processing digits
Processing mnist
Done!


# Examine the training data

In [4]:
len(train_set)

697932

In [5]:
print('Max Label id: ',train_set.train_labels.max().item())
print('Min Label id: ',train_set.train_labels.min().item())
print('Images in each label: ',train_set.train_labels.bincount())



Max Label id:  61
Min Label id:  0
Images in each label:  tensor([34585, 38374, 34203, 35143, 33535, 31416, 34232, 35754, 33946, 33847,  6407,  3878, 10094,  4562,  4934,  9182,
         2517,  3152, 11946,  3762,  2468,  5076,  9002,  8237, 24983,  8347,  2605,  5073, 20764,  9820, 12602,  4637,
         4695,  2771,  4743,  2701, 10033,  5159,  2854, 10177, 24631,  2561,  3687,  8738,  2725,  1896,  2491, 15318,
         2645, 11418,  2749,  2448,  2994, 14105,  2699, 18262,  2830,  2910,  2697,  2822,  2365,  2725])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [6]:
sample = next(iter(train_set))
image, label = sample 

image.shape, label

(torch.Size([1, 28, 28]), 35)

# Design the training model

In [7]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=30, out_channels=62, kernel_size=3, padding=1)


    def forward(self, t):                                           #input size >> output size >> receptive field
        # (1) Input layer
        t=t                                                         #1x28x28 >> 1x28x28 >> 0

        # (2) hidden conv layer
        t=self.conv1(t)                                             #1x28x28 >> 10x28x28 >> 3
        t=F.relu(t)    

        # (3) hidden conv layer
        t=self.conv2(t)                                             #10x28x28 >> 10x28x28 >> 5 
        t=F.relu(t)
        t=F.max_pool2d(t, kernel_size=2, stride=2)                  #10x28x28 >> 10x14x14 >> 10

        # (4) hidden conv layer
        t=self.conv3(t)                                             #10x14x14 >> 20x14x14 >> 12
        t=F.relu(t)    

        # (5) hidden conv layer
        t=self.conv4(t)                                             #20x14x14 >> 20x14x14 >> 14
        t=F.relu(t)
        t=F.max_pool2d(t, kernel_size=2, stride=2)                  #20x14x14 >> 20x7x7 >> 28

        # (6) hidden conv layer
        t=self.conv5(t)                                             #20x7x7 >> 30x7x7 >> 30
        t=F.relu(t)    

        # (7) hidden conv layer
        t=self.conv6(t)                                             #30x7x7 >> 62x7x7 >> 32
        t=F.relu(t)

        # (8) output layer
        t=F.adaptive_avg_pool2d(t, (1, 1))                          #62x7x7 >> 62x1x1 
        t=t.squeeze()                                               #62x1x1 >> 62
        return t





# Verifying the Network output

In [8]:
network1 = Network()

image = image.reshape(1,1,28,28)
t1 = network1(image)
print(t1.shape)
t1

torch.Size([62])


tensor([0.0000e+00, 1.8094e-02, 2.5436e-02, 0.0000e+00, 4.1494e-02, 8.9119e-04, 0.0000e+00, 0.0000e+00, 6.8858e-02,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.4216e-03, 0.0000e+00, 0.0000e+00,
        5.0215e-02, 0.0000e+00, 1.8835e-03, 0.0000e+00, 0.0000e+00, 1.5910e-02, 0.0000e+00, 1.0536e-02, 3.0548e-02,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 6.0819e-02, 0.0000e+00, 3.5168e-02, 3.9394e-03, 1.4906e-04, 3.9383e-02,
        2.7945e-02, 3.4765e-05, 1.5398e-02, 2.9058e-02, 8.7894e-05, 3.2181e-02, 2.3391e-02, 0.0000e+00, 1.1552e-02,
        8.6316e-03, 0.0000e+00, 2.5356e-05, 0.0000e+00, 3.2607e-02, 0.0000e+00, 0.0000e+00, 2.0887e-02, 5.0428e-02,
        0.0000e+00, 3.6003e-03, 6.0142e-02, 0.0000e+00, 5.5059e-02, 4.4658e-02, 0.0000e+00, 0.0000e+00],
       grad_fn=<SqueezeBackward0>)

In [9]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

# Training the neural network 

To train a network, need to follow the following process

1. create a batch of images
2. calculate the loss function(cross entropy loss function)
3. calculate the gradient(backprop)
4. update the weights to reduce the loss(optimizer)
5. repeat step 1 to 4 for all the batches untill you reach 1 epoch

In [10]:
network = Network()

optimizer=optim.Adam(network.parameters(), lr=0.01)  # Using adam optimizer

for epoch in range(20):      # 1 epoch is training the network through all the images once, we have 20 epochs here 
    
    total_loss =0
    total_correct=0

    for batch in train_loader:
        images, labels = batch

        preds = network(images) # pass batch of images through the network
        loss=F.cross_entropy(preds, labels) # calculate the loss

        # pytorch accumulates the gradients
        # so zero out the gradients before calculating again
        optimizer.zero_grad() 
        loss.backward() # calculate gradient
        optimizer.step() # update weights

        total_loss += loss
        total_correct += get_num_correct(preds,labels)

    print("epoch:", epoch, "total correct:", total_correct, 'total loss:', total_loss)




epoch: 0 total correct: 389753 total loss: tensor(13548.6113, grad_fn=<AddBackward0>)
epoch: 1 total correct: 406012 total loss: tensor(12671.5264, grad_fn=<AddBackward0>)
epoch: 2 total correct: 407602 total loss: tensor(12584.1641, grad_fn=<AddBackward0>)
epoch: 3 total correct: 408396 total loss: tensor(12516.3193, grad_fn=<AddBackward0>)
epoch: 4 total correct: 408811 total loss: tensor(12471.5908, grad_fn=<AddBackward0>)
epoch: 5 total correct: 409368 total loss: tensor(12438.0645, grad_fn=<AddBackward0>)
epoch: 6 total correct: 409385 total loss: tensor(12439.2246, grad_fn=<AddBackward0>)
epoch: 7 total correct: 409802 total loss: tensor(12428.3203, grad_fn=<AddBackward0>)
epoch: 8 total correct: 410131 total loss: tensor(12398.0547, grad_fn=<AddBackward0>)
epoch: 9 total correct: 410357 total loss: tensor(12387.2148, grad_fn=<AddBackward0>)
epoch: 10 total correct: 410334 total loss: tensor(12382.0547, grad_fn=<AddBackward0>)
epoch: 11 total correct: 410634 total loss: tensor(12

In [11]:
#Accuracy
total_correct/len(train_set)

0.5886690967028306